# Energy bookkeeping tests for `HybridTopologyFactory`

In [1]:
import copy
from simtk import openmm, unit
from simtk.openmm import app
import os, os.path
import sys, math
import numpy as np
from openeye import oechem
from openmmtools.constants import kB
from openmmtools import alchemy, states

temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
CARBON_MASS = 12.01

## The first test is a valence-only vacuum transformation from a 3-4 bead system

The first thing to do is generating the system...

In [ ]:
from perses.tests.test_geometry_engine_analytical import AnalyticalBeadSystems
ABS = AnalyticalBeadSystems([4,5], 1)
nbfA, nbfB = openmm.NonbondedForce(), openmm.NonbondedForce()


We are going to make nonbonded interactions for each and every particle in the system for bookkeeping purposes...

In [ ]:
nbf = openmm.NonbondedForce()
nbf.setNonbondedMethod(0)
for _ in range(4):
    a = nbf.addParticle(1., 0.1, 1.)
    print(a)
ABS.sys_pos_top['A'][0].addForce(nbf)

In [ ]:
nbf = openmm.NonbondedForce()
nbf.setNonbondedMethod(0)
for _ in range(5):
    a = nbf.addParticle(1., 0.1, 1.)
    print(a)
    
#add exception
nbf.addException(0,4, 4., 0.3, 2.)
ABS.sys_pos_top['B'][0].addForce(nbf)

In [ ]:
ABS.sys_pos_top['A'][0].getForces()

Now we can generate the topology proposal...

In [ ]:
top_proposal = ABS.create_simple_topology_proposal(ABS.sys_pos_top, 4, 5)

Now we have to make the topology proposal from 3 to 4


In [ ]:
from perses.rjmc.geometry import FFAllAngleGeometryEngine
_geometry_engine = FFAllAngleGeometryEngine(metadata=None, use_sterics=False, n_bond_divisions=100, n_angle_divisions=180, n_torsion_divisions=360, verbose=True, storage=None, bond_softening_constant=1.0, angle_softening_constant=1.0, neglect_angles = False)
_new_positions, _lp = _geometry_engine.propose(top_proposal, ABS.sys_pos_top['A'][1], beta)


Now that we have the proposal and the new positions, we can try to build a hybrid system


In [ ]:
from perses.annihilation.new_relative import HybridTopologyFactory
HTF = HybridTopologyFactory(top_proposal, ABS.sys_pos_top['A'][1], _new_positions)

In [ ]:
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import copy
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
hybrid_system = HTF.hybrid_system
lambda_zero_alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
lambda_one_alchemical_state = copy.deepcopy(lambda_zero_alchemical_state)

lambda_zero_alchemical_state.set_alchemical_parameters(0.0)
lambda_one_alchemical_state.set_alchemical_parameters(1.0)

In [ ]:
# Create thermodynamic states for the nonalchemical endpoints
thermodynamic_state = ThermodynamicState(hybrid_system, temperature = temperature)
nonalchemical_thermodynamic_states = {
    0: ThermodynamicState(top_proposal.old_system, temperature=temperature),
    1: ThermodynamicState(top_proposal.new_system, temperature=temperature)}

# Now create the compound states with different alchemical states
hybrid_thermodynamic_states = {0: CompoundThermodynamicState(thermodynamic_state,
                                                                   composable_states=[lambda_zero_alchemical_state]),
                                     1: CompoundThermodynamicState(copy.deepcopy(thermodynamic_state),
                                                                   composable_states=[lambda_one_alchemical_state])}


In [ ]:
#create contexts
from openmmtools import integrators
from perses.tests.utils import compute_potential_components
nonalchemical_zero_context = nonalchemical_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_zero_context.setPositions(ABS.sys_pos_top['A'][1])
nonalchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_zero_context)]
print(f"nonalchemical_zero_energy components")
for force, energy in nonalchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_zero_energy: {sum(nonalchemical_zero_energy_components)}")
print()

nonalchemical_one_context = nonalchemical_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_one_context.setPositions(_new_positions)
nonalchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_one_context)]
print(f"nonalchemical_one_energy components")
for force, energy in nonalchemical_one_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_one_energy: {sum(nonalchemical_one_energy_components)}")
print()
print()

alchemical_zero_context = hybrid_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_zero_context.setPositions(HTF._hybrid_positions)
alchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_zero_context)]
print(f"alchemical_zero_energy components")
for force, energy in alchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"alchemical_zero_energy: {sum(alchemical_zero_energy_components)}")
print()

alchemical_one_context = hybrid_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_one_context.setPositions(HTF._hybrid_positions)
alchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_one_context)]
print(f"alchemical_one_energy components")
for force, energy in alchemical_one_energy_components:
    print(f"\t{force}: {energy}")
#print(f"alchemical_one_energy: {sum(alchemical_one_energy_components)}")

## Next test is to conduct a vacuum transformation for some small molecules and test that the $\Delta U$ between the alchemical and nonalchemical systems is just the valence terms added and subtracted

In [ ]:
from perses.tests.utils import generate_vacuum_topology_proposal
from perses.rjmc.geometry import FFAllAngleGeometryEngine

In [ ]:
top_proposal, old_positions, new_positions = generate_vacuum_topology_proposal('benzene', 'toluene')

In [ ]:
_geometry_engine = FFAllAngleGeometryEngine(metadata=None, use_sterics=False, n_bond_divisions=100, n_angle_divisions=180, n_torsion_divisions=360, verbose=True, storage=None, bond_softening_constant=1.0, angle_softening_constant=1.0, neglect_angles = False)

In [ ]:
_new_positions, _lp = _geometry_engine.propose(top_proposal, old_positions, beta)

In [ ]:
_lp_rev = _geometry_engine.logp_reverse(top_proposal, _new_positions, old_positions, beta)

In [ ]:
from perses.annihilation.new_relative import HybridTopologyFactory
HTF = HybridTopologyFactory(top_proposal, old_positions, _new_positions)

In [ ]:
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import copy
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
from openmmtools import integrators

hybrid_system = HTF.hybrid_system
lambda_zero_alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
lambda_one_alchemical_state = copy.deepcopy(lambda_zero_alchemical_state)

lambda_zero_alchemical_state.set_alchemical_parameters(0.0)
lambda_one_alchemical_state.set_alchemical_parameters(1.0)




In [ ]:
# Create thermodynamic states for the nonalchemical endpoints
thermodynamic_state = ThermodynamicState(hybrid_system, temperature = temperature)
nonalchemical_thermodynamic_states = {
    0: ThermodynamicState(top_proposal.old_system, temperature=temperature),
    1: ThermodynamicState(top_proposal.new_system, temperature=temperature)}

# Now create the compound states with different alchemical states
hybrid_thermodynamic_states = {0: CompoundThermodynamicState(thermodynamic_state,
                                                                   composable_states=[lambda_zero_alchemical_state]),
                                     1: CompoundThermodynamicState(copy.deepcopy(thermodynamic_state),
                                                                   composable_states=[lambda_one_alchemical_state])}

In [ ]:



#create contexts
from openmmtools import integrators
from perses.tests.utils import compute_potential_components
nonalchemical_zero_context = nonalchemical_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_zero_context.setPositions(old_positions)
nonalchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_zero_context)]
print(f"nonalchemical_zero_energy components")
for force, energy in nonalchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_zero_energy: {sum(nonalchemical_zero_energy_components)}")
print()

nonalchemical_one_context = nonalchemical_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_one_context.setPositions(_new_positions)
nonalchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_one_context)]
print(f"nonalchemical_one_energy components")
for force, energy in nonalchemical_one_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_one_energy: {sum(nonalchemical_one_energy_components)}")
print()
print()

alchemical_zero_context = hybrid_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_zero_context.setPositions(HTF._hybrid_positions)
alchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_zero_context)]
print(f"alchemical_zero_energy components")
for force, energy in alchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"alchemical_zero_energy: {sum(alchemical_zero_energy_components)}")
print()

alchemical_one_context = hybrid_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_one_context.setPositions(HTF._hybrid_positions)
alchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_one_context)]
print(f"alchemical_one_energy components")
for force, energy in alchemical_one_energy_components:
    print(f"\t{force}: {energy}")

In [ ]:
forward_added_valence_energy = _geometry_engine.forward_final_context_reduced_potential - _geometry_engine.forward_atoms_with_positions_reduced_potential
reverse_subtracted_valence_energy = _geometry_engine.reverse_final_context_reduced_potential - _geometry_engine.reverse_atoms_with_positions_reduced_potential

In [ ]:
nonalch_zero_rp = SamplerState.from_context(nonalchemical_zero_context).potential_energy*beta
nonalch_one_rp = SamplerState.from_context(nonalchemical_one_context).potential_energy*beta

alch_zero_rp = SamplerState.from_context(alchemical_zero_context).potential_energy*beta
alch_one_rp = SamplerState.from_context(alchemical_one_context).potential_energy*beta

In [ ]:
print(f"nonalchemical_zero: {nonalch_zero_rp}, alch_zero - forward valence: {alch_zero_rp - forward_added_valence_energy}")

In [ ]:
print(f"nonalchemical_one: {nonalch_one_rp}, alch_one - reverse valence: {alch_one_rp - reverse_subtracted_valence_energy}")

In [ ]:
old_positions.value_in_unit(unit.nanometers).astype(np.float64)

## Last test is to conduct a `SOLVATED` transformation for some small molecules and test that the $\Delta U$ between the alchemical and nonalchemical systems is just the valence terms added and subtracted.

In [2]:
from perses.tests.utils import generate_solvated_hybrid_test_topology
from perses.rjmc.geometry import FFAllAngleGeometryEngine

In [3]:
#first, try it as a vacuum...so we treat environment atoms with the same nonbonded method as we did before...

In [4]:
#i suspect there may be a discrepancy in the way the hybrid topology factory sets the nonbonded method

In [5]:
top_proposal, old_positions= generate_solvated_hybrid_test_topology('benzene', 'toluene', propose_geometry = False)

DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:proposal_engine:Building new Topology object...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:proposal_engine:Proposed transformation would delete 1 atoms and create 4 atoms.


In [6]:
# top_proposal._old_system.getForce(3).setNonbondedMethod(0)
# top_proposal._new_system.getForce(3).setNonbondedMethod(0)

# top_proposal._old_system.removeForce(4)
# top_proposal._new_system.removeForce(4)

In [7]:
top_proposal._old_system.getForce(3).setUseDispersionCorrection(False)
top_proposal._new_system.getForce(3).setUseDispersionCorrection(False)

In [8]:
_geometry_engine = FFAllAngleGeometryEngine(metadata=None, use_sterics=False, n_bond_divisions=100, n_angle_divisions=180, n_torsion_divisions=360, verbose=True, storage=None, bond_softening_constant=1.0, angle_softening_constant=1.0, neglect_angles = False)

In [9]:
old_positions.value_in_unit(unit.nanometers)

array([[ 0.1885141 , -0.10363065, -0.01124404],
       [ 0.08065562, -0.04630106,  0.05610063],
       [ 0.29210052, -0.16308655,  0.06081435],
       ...,
       [ 0.08351727,  0.62023201,  0.09468868],
       [ 0.11641727,  0.66563201,  0.01708868],
       [ 0.15791727,  0.61833201,  0.15478868]])

In [10]:
_new_positions, _lp = _geometry_engine.propose(top_proposal, old_positions, beta)

INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index proposal order is [1125, 1131, 1132, 1133]
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 15 bonds in reference force.
INFO:geometry:		for bond 0 (i.e. partices 1119 and 1120), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for bond 1 (i.e. partices 1119 and 1121), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for bond 2 (i.e. partices 1120 and 1122), the growth_index is 0
INFO:geometry:			adding to th

INFO:geometry:			adding to the growth system
INFO:geometry:		for torsion 12 (i.e. partices 1122, 1120, 1119, and 1126), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 13 (i.e. partices 1122, 1124, 1123, and 1130), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 14 (i.e. partices 1123, 1121, 1119, and 1126), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 15 (i.e. partices 1123, 1124, 1122, and 1129), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 16 (i.e. partices 1124, 1122, 1120, and 1127), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 17 (i.e. partices 1124, 1123, 1121, and 1128), the growth_index is 0
INFO:geometry:			adding to the the atoms with positio

In [11]:
_lp_rev = _geometry_engine.logp_reverse(top_proposal, _new_positions, old_positions, beta)

INFO:geometry:logp_reverse: performing reverse proposal
INFO:geometry:logp_reverse: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index proposal order is [11]
INFO:geometry:direction of proposal is reverse; creating atoms_with_positions from old system/topology
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 12 bonds in reference force.
INFO:geometry:		for bond 0 (i.e. partices 0 and 1), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for bond 1 (i.e. partices 0 and 2), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for bond 2 (i.e. partices 1 and 3), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:

INFO:geometry:			adding to the growth system
INFO:geometry:		for torsion 23 (i.e. partices 10, 4, 5, and 11), the growth_index is 1
INFO:geometry:			adding to the growth system
INFO:geometry:		for torsion 24 (i.e. partices 1, 2, 0, and 6), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 25 (i.e. partices 0, 3, 1, and 7), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 26 (i.e. partices 0, 4, 2, and 8), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 27 (i.e. partices 1, 5, 3, and 9), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 28 (i.e. partices 2, 5, 4, and 10), the growth_index is 0
INFO:geometry:			adding to the the atoms with positions system.
INFO:geometry:		for torsion 29 (i.e. partices 3, 4, 5, and 11), the growth_in

In [12]:
from perses.annihilation.new_relative import HybridTopologyFactory
HTF = HybridTopologyFactory(top_proposal, old_positions, _new_positions)

INFO:new_relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:new_relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:new_relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:new_relative:No unknown forces.
INFO:new_relative:Nonbonded method to be used (i.e. from old system): 4
INFO:new_relative:Adding and mapping old atoms to hybrid system...
INFO:new_relative:Adding and mapping new atoms to hybrid system...
INFO:new_relative:Added MonteCarloBarostat.
INFO:new_relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=(2.2959172248840334, 0.0, 0.0), unit=nanometer), Quantity(value=(0.0, 2.2959172248840334, 0.0), unit=nanometer), Quantity(value=(0.0, 0.0, 2.2959172248840334), unit=nanometer)]
INFO:new_relative:Determined atom classes.
I

INFO:new_relative:Generating new system exceptions dict...
INFO:new_relative:	_generate_dict_from_exceptions: Exceptions Dict: {(0, 1): [Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)], (0, 2): [Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)], (1, 2): [Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)], (3, 4): [Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)], (3, 5): [Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)], (4, 5): [Quantity(value=0.0, unit=elementary charge**2), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)], (6, 7): [Quantity(value=0.0, unit=elementary charg

INFO:new_relative:Handling constraints...
INFO:new_relative:	_handle_constraints: constraint_lengths dict: {(12, 13): Quantity(value=0.09572, unit=nanometer), (12, 14): Quantity(value=0.09572, unit=nanometer), (15, 16): Quantity(value=0.09572, unit=nanometer), (15, 17): Quantity(value=0.09572, unit=nanometer), (18, 19): Quantity(value=0.09572, unit=nanometer), (18, 20): Quantity(value=0.09572, unit=nanometer), (21, 22): Quantity(value=0.09572, unit=nanometer), (21, 23): Quantity(value=0.09572, unit=nanometer), (24, 25): Quantity(value=0.09572, unit=nanometer), (24, 26): Quantity(value=0.09572, unit=nanometer), (27, 28): Quantity(value=0.09572, unit=nanometer), (27, 29): Quantity(value=0.09572, unit=nanometer), (30, 31): Quantity(value=0.09572, unit=nanometer), (30, 32): Quantity(value=0.09572, unit=nanometer), (33, 34): Quantity(value=0.09572, unit=nanometer), (33, 35): Quantity(value=0.09572, unit=nanometer), (36, 37): Quantity(value=0.09572, unit=nanometer), (36, 38): Quantity(value=

INFO:new_relative:Handling virtual sites...
INFO:new_relative:	_handle_virtual_sites: numVirtualSites: 0
INFO:new_relative:Adding bond force terms...
INFO:new_relative:Adding angle force terms...
INFO:new_relative:Adding torsion force terms...
INFO:new_relative:Adding nonbonded force terms...
INFO:new_relative:	_add_nonbonded_force_terms: <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x000001F08BEBE4E0> > added to hybrid system
INFO:new_relative:	_add_nonbonded_force_terms: nonbonded_method is PME or Ewald
INFO:new_relative:	_add_nonbonded_force_terms: 4 added to standard nonbonded force
INFO:new_relative:	_add_nonbonded_force_terms: 2 added to sterics_custom_nonbonded force
INFO:new_relative:	_add_nonbonded_force_terms: <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x000001F08BEBE600> > added to hybrid system
INFO:new_relative:Handling harmonic bonds...
INFO:new_relative:	

INFO:new_relative:		handle_harmonic_angles: angle_index 1 is a core (to custom angle force).
INFO:new_relative:		handle_harmonic_angles: new angle_index: 2
INFO:new_relative:		handle_harmonic_angles: angle_index 2 is a core (to custom angle force).
INFO:new_relative:		handle_harmonic_angles: new angle_index: 3
INFO:new_relative:		handle_harmonic_angles: angle_index 3 is a core (to custom angle force).
INFO:new_relative:		handle_harmonic_angles: new angle_index: 4
INFO:new_relative:		handle_harmonic_angles: angle_index 4 is a core (to custom angle force).
INFO:new_relative:		handle_harmonic_angles: new angle_index: 5
INFO:new_relative:		handle_harmonic_angles: angle_index 5 is a core (to custom angle force).
INFO:new_relative:		handle_harmonic_angles: new angle_index: 6
INFO:new_relative:		handle_harmonic_angles: angle_index 6 is a core (to custom angle force).
INFO:new_relative:		handle_harmonic_angles: new angle_index: 7
INFO:new_relative:		handle_harmonic_angles: angle_index 7 is a c

...

INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (918, 919) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (918, 920) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (919, 920) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (921, 922) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (921, 923) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (922, 923) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (924, 925) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (924, 926) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (925, 926) is an environment exception pair
I

INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (991, 992) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (993, 994) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (993, 995) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (994, 995) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (996, 997) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (996, 998) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (997, 998) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (999, 1000) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (999, 1001) is an environment exception pair

INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1065, 1066) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1065, 1067) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1066, 1067) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1068, 1069) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1068, 1070) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1069, 1070) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1071, 1072) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1071, 1073) is an environment exception pair
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1072, 1073) is an environmen

INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1120, 1123) is a core-core exception pair.
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1121, 1123) is a core-core exception pair.
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1122, 1123) is a core-core exception pair.
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1119, 1124) is a core-core exception pair.
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1120, 1124) is a core-core exception pair.
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1121, 1124) is a core-core exception pair.
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1122, 1124) is a core-core exception pair.
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1123, 1124) is a core-core exception pair.
INFO:new_relative:		handle_nonbonded: _handle_original_exceptions: (1120, 1125) is a unique_new-core or unique_n

INFO:new_relative:		handle_old_new_exceptions: (1123, 1125) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1124, 1125) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1125, 1129) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1125, 1130) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1122, 1131) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1123, 1131) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1124, 1131) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1125, 1131) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1122, 1132) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1123, 1132) is a unique_new exception pair.
INFO:new_relative:		handle_old_new_exceptions: (1124, 1132) is a unique_new exce

In [14]:
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import copy
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
from openmmtools import integrators

hybrid_system = HTF.hybrid_system
lambda_zero_alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
lambda_one_alchemical_state = copy.deepcopy(lambda_zero_alchemical_state)

lambda_zero_alchemical_state.set_alchemical_parameters(0.0)
lambda_one_alchemical_state.set_alchemical_parameters(1.0)

hybrid_system.getForce(7).setUseDispersionCorrection(False)

In [15]:
# Create thermodynamic states for the nonalchemical endpoints
thermodynamic_state = ThermodynamicState(hybrid_system, temperature = temperature)
nonalchemical_thermodynamic_states = {
    0: ThermodynamicState(top_proposal.old_system, temperature=temperature),
    1: ThermodynamicState(top_proposal.new_system, temperature=temperature)}

# Now create the compound states with different alchemical states
hybrid_thermodynamic_states = {0: CompoundThermodynamicState(thermodynamic_state,
                                                                   composable_states=[lambda_zero_alchemical_state]),
                                     1: CompoundThermodynamicState(copy.deepcopy(thermodynamic_state),
                                                                   composable_states=[lambda_one_alchemical_state])}

In [16]:
#create contexts
from openmmtools import integrators
from perses.tests.utils import compute_potential_components
nonalchemical_zero_context = nonalchemical_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_zero_context.setPositions(old_positions)
nonalchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_zero_context)]
print(f"nonalchemical_zero_energy components")
for force, energy in nonalchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_zero_energy: {sum(nonalchemical_zero_energy_components)}")
print()

nonalchemical_one_context = nonalchemical_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_one_context.setPositions(_new_positions)
nonalchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_one_context)]
print(f"nonalchemical_one_energy components")
for force, energy in nonalchemical_one_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_one_energy: {sum(nonalchemical_one_energy_components)}")
print()
print()

alchemical_zero_context = hybrid_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_zero_context.setPositions(HTF._hybrid_positions)
alchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_zero_context)]
print(f"alchemical_zero_energy components")
for force, energy in alchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"alchemical_zero_energy: {sum(alchemical_zero_energy_components)}")
print()

alchemical_one_context = hybrid_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_one_context.setPositions(HTF._hybrid_positions)
alchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_one_context)]
print(f"alchemical_one_energy components")
for force, energy in alchemical_one_energy_components:
    print(f"\t{force}: {energy}")

nonalchemical_zero_energy components
	HarmonicBondForce: 0.05787802063460886
	HarmonicAngleForce: 0.004459541747343244
	PeriodicTorsionForce: 0.00011703901332337944
	NonbondedForce: 593.0539160343947
	MonteCarloBarostat: 0.0

nonalchemical_one_energy components
	HarmonicBondForce: 1.2416959908592518
	HarmonicAngleForce: 10.310224820826036
	PeriodicTorsionForce: 0.37758687573759003
	NonbondedForce: 602.0738757497379
	MonteCarloBarostat: 0.0


alchemical_zero_energy components
	CustomBondForce: 0.05780943147099217
	HarmonicBondForce: 1.1839551485518764
	CustomAngleForce: 0.003730209639256297
	HarmonicAngleForce: 10.307223943294865
	CustomTorsionForce: 0.00011323521200709491
	PeriodicTorsionForce: 0.3774774443268991
	NonbondedForce: 602.7663547531444
	CustomNonbondedForce: -11.407151114482774
	CustomBondForce: 1.6947123957220158
	MonteCarloBarostat: 0.0

alchemical_one_energy components
	CustomBondForce: 0.05780943147099217
	HarmonicBondForce: 1.1839551485518764
	CustomAngleForce: 0.00373

In [17]:
forward_added_valence_energy = _geometry_engine.forward_final_context_reduced_potential - _geometry_engine.forward_atoms_with_positions_reduced_potential
reverse_subtracted_valence_energy = _geometry_engine.reverse_final_context_reduced_potential - _geometry_engine.reverse_atoms_with_positions_reduced_potential

In [18]:
nonalch_zero_rp = SamplerState.from_context(nonalchemical_zero_context).potential_energy*beta
nonalch_one_rp = SamplerState.from_context(nonalchemical_one_context).potential_energy*beta

alch_zero_rp = SamplerState.from_context(alchemical_zero_context).potential_energy*beta
alch_one_rp = SamplerState.from_context(alchemical_one_context).potential_energy*beta

In [19]:
print(f"nonalchemical_zero: {nonalch_zero_rp}, alch_zero - forward valence: {alch_zero_rp - forward_added_valence_energy}")

nonalchemical_zero: 593.0911796288584, alch_zero - forward valence: 593.0940409537641


In [20]:
print(f"nonalchemical_one: {nonalch_one_rp}, alch_one - reverse valence: {alch_one_rp - reverse_subtracted_valence_energy}")

nonalchemical_one: 613.9801826542063, alch_one - reverse valence: 613.9810619230802


In [ ]:
hybrid_system.getForce(7).getUseDispersionCorrection()

In [ ]:
top_proposal._old_system.getForce(3).getUseDispersionCorrection()

In [ ]:
top_proposal._new_system.getForce(3).getUseDispersionCorrection()